In [1]:
import pandas as pd

In [2]:
df=pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [3]:
df = df.set_index(pd.DatetimeIndex(df['Created Date']))

del df['Created Date']

In [4]:
#Q1

In [5]:
df.index = pd.to_datetime(df.index)

In [6]:
df_2022 = df[df.index.year == 2022]

In [7]:
average_daily_complaints_2022 = df_2022['Unique Key'].resample('D').count().mean()

In [8]:
print(f"The average number of daily complaints received in 2022 is: {average_daily_complaints_2022:.1f}")

The average number of daily complaints received in 2022 is: 8684.3


In [9]:
#Q2

In [10]:
daily_call_count = df['Unique Key'].resample('D').count()

In [11]:
max_calls_date = daily_call_count.idxmax()

In [12]:
print(f"The date with the maximum number of calls is: {max_calls_date}")

The date with the maximum number of calls is: 2020-08-04 00:00:00


In [13]:
#Q3

In [14]:
max_calls_data = df[df.index.date == max_calls_date.date()]

In [15]:
most_common_complaint_type = max_calls_data['Complaint Type'].mode().iloc[0]

In [16]:
print(f"On the date with the maximum number of calls ({max_calls_date}), the most important complaint type was: {most_common_complaint_type}")

On the date with the maximum number of calls (2020-08-04 00:00:00), the most important complaint type was: Damaged Tree


In [17]:
#Q4

In [18]:
monthly_call_count = df.groupby(df.index.month)['Unique Key'].count()

In [19]:
quietest_month = monthly_call_count.idxmin()

In [20]:
print(f"The historically quietest month is: {quietest_month}")

The historically quietest month is: 12


In [21]:
#Q5

In [22]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

In [27]:
daily_series = df['Unique Key'].resample('D').count()

In [28]:
result = seasonal_decompose(daily_series, model = 'additive') 

In [29]:
seasonal_component = result.seasonal['2020-12-25'].round()
print(f"Seasonal component on 2020-12-25: {seasonal_component}")

Seasonal component on 2020-12-25: 183.0


In [30]:
#Q6

In [31]:
autocorrelation_lag_1 = daily_call_count.autocorr(lag=1)

print(f"The autocorrelation with the number of calls the day prior (lag of 1) is: {autocorrelation_lag_1:.2f}")

The autocorrelation with the number of calls the day prior (lag of 1) is: 0.75


In [32]:
#Q7

In [33]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np

In [34]:
df_prophet = pd.DataFrame({'ds': daily_call_count.index, 'y': daily_call_count.values})
train_size = int(len(df_prophet) - 90)
train, test = df_prophet[0:train_size], df_prophet[train_size:]
model = Prophet()
model.fit(train)
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)
y_true = test['y'].values
y_pred = forecast.iloc[-90:]['yhat'].values
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"The RMSE on the test set is: {rmse:.1f}")

15:28:42 - cmdstanpy - INFO - Chain [1] start processing
15:28:43 - cmdstanpy - INFO - Chain [1] done processing


The RMSE on the test set is: 1231.5
